In [14]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

File 'edgar_allan_poe.txt' already there; not retrieving.

File 'robert_frost.txt' already there; not retrieving.



In [15]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [16]:
input_files = [
  'edgar_allan_poe.txt',
  'robert_frost.txt',
]

In [25]:
with open('edgar_allan_poe.txt', 'rb') as f:
    for _ in range(10):
        print(f.readline().decode('utf-8'))

LO! Death hath rear'd himself a throne

In a strange city, all alone,

Far down within the dim west

Where the good, and the bad, and the worst, and the best,

Have gone to their eternal rest.

 

There shrines, and palaces, and towers

Are not like any thing of ours

Oh no! O no! ours never loom

To heaven with that ungodly gloom!



In [23]:
with open('robert_frost.txt', 'rb') as f:
    for _ in range(10):
        print(f.readline().decode('utf-8'))

Two roads diverged in a yellow wood,

And sorry I could not travel both

And be one traveler, long I stood

And looked down one as far as I could

To where it bent in the undergrowth; 



Then took the other, as just as fair,

And having perhaps the better claim

Because it was grassy and wanted wear,

Though as for that the passing there



In [24]:
# collect data into lists
input_texts = []
labels = []

for label, f in enumerate(input_files):
  print(f"{f} corresponds to label {label}")

  for line in open(f, 'r', encoding='UTF-8'):
    line = line.rstrip().lower()
    if line:
      # remove punctuation
      line = line.translate(str.maketrans('', '', string.punctuation))

      input_texts.append(line)
      labels.append(label)

edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


In [26]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [27]:
len(Ytrain), len(Ytest)

(1615, 539)

In [28]:
train_text[:5]

['i cant keep track of other peoples daughters',
 'whod marry her straight out of such a mess',
 'someday i will',
 'and i said what is written sweet sister',
 'i heard them toffile didnt seem to hear them']

In [29]:
Ytrain[:5]

[1, 1, 1, 0, 1]

In [30]:
idx = 1
word2idx = {'<unk>': 0}

In [31]:
# tokenize and populate word2idx
for text in train_text:
    tokens = text.split()
    for token in tokens:
      if token not in word2idx:
        word2idx[token] = idx
        idx += 1

In [32]:
word2idx

{'<unk>': 0,
 'i': 1,
 'cant': 2,
 'keep': 3,
 'track': 4,
 'of': 5,
 'other': 6,
 'peoples': 7,
 'daughters': 8,
 'whod': 9,
 'marry': 10,
 'her': 11,
 'straight': 12,
 'out': 13,
 'such': 14,
 'a': 15,
 'mess': 16,
 'someday': 17,
 'will': 18,
 'and': 19,
 'said': 20,
 'what': 21,
 'is': 22,
 'written': 23,
 'sweet': 24,
 'sister': 25,
 'heard': 26,
 'them': 27,
 'toffile': 28,
 'didnt': 29,
 'seem': 30,
 'to': 31,
 'hear': 32,
 'the': 33,
 'rural': 34,
 'letterbox': 35,
 'lajway': 36,
 'in': 37,
 'rain': 38,
 'tomorrow': 39,
 'shall': 40,
 'we': 41,
 'if': 42,
 'it': 43,
 'rains': 44,
 'let': 45,
 'myself': 46,
 'at': 47,
 'kitchen': 48,
 'door': 49,
 'save': 50,
 'for': 51,
 'wonder': 52,
 'paddle': 53,
 'blade': 54,
 'people': 55,
 'than': 56,
 'can': 57,
 'yankees': 58,
 'are': 59,
 'they': 60,
 'always': 61,
 'were': 62,
 'gushing': 63,
 'strange': 64,
 'with': 65,
 'tears': 66,
 'ah': 67,
 'safely': 68,
 'may': 69,
 'trust': 70,
 'its': 71,
 'gleaming': 72,
 'but': 73,
 'had': 

In [33]:
len(word2idx)

2511

In [34]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token, 0) for token in tokens] # some words may not presented in train
  test_text_int.append(line_as_int)

In [35]:
train_text_int[100:105]

[[77, 385, 335, 373, 1, 57, 230, 138],
 [78, 1, 386, 15, 387, 388, 343, 91],
 [389, 94, 77, 390, 33, 391, 99, 11, 392],
 [37, 78, 393, 394],
 [120, 395, 396, 397, 398, 141, 399]]

In [36]:
# initialize A(state transition probability) and pi(initial state probability) matrices - for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [37]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sentence
        pi[idx] += 1
      else:
        # the last word exists, so count a transition
        A[last_idx, idx] += 1

      # update last idx
      last_idx = idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [38]:
# normalize A and pi so they are valid probability matrices
# convince yourself that this is equivalent to the formulas shown before
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [39]:
# log A and pi since we don't need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [40]:
# compute priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.33126934984520123, 0.6687306501547987)

In [41]:
# build a classifier
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # it's the first token
        logprob += logpi[idx]
      else:
        logprob += logA[last_idx, idx]
      
      # update last_idx
      last_idx = idx
    
    return logprob
  
  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
             for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [42]:
# each array must be in order since classes are assumed to index these lists
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [43]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.993188854489164


In [45]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.8033395176252319


In [46]:
from sklearn.metrics import confusion_matrix, f1_score

In [47]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[ 524,   11],
       [   0, 1080]], dtype=int64)

In [48]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[ 82, 101],
       [  5, 351]], dtype=int64)

In [49]:
f1_score(Ytrain, Ptrain)

0.9949332105020728

In [50]:
f1_score(Ytest, Ptest)

0.8688118811881188